In [139]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from scrape_pro_football_reference import get_player_df

In [140]:
df = get_player_df(2020)
df.dropna(inplace=True)
next_year_PPR = get_player_df(2021)[["Player", "PPR"]]
next_year_PPR.dropna(inplace=True)
df.head()

,Player,Tm,FantPos,Age,G,GS,Cmp,Att,Yds,TD,Int,Att,Yds,Y/A,TD,Tgt,Rec,Yds,Y/R,TD,Fmb,FL,TD,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank
0,Derrick Henry *+,TEN,RB,26,16,16,0,0,0,0,0,378,2027,5.36,17,31,19,114,6.00,0,3,2,17,1,,314,333.1,341.1,323.6,184,1,1
1,Alvin Kamara*,NOR,RB,25,15,10,0,0,0,0,0,187,932,4.98,16,107,83,756,9.11,5,1,0,21,,,295,377.8,383.8,336.3,165,2,2
2,Dalvin Cook*,MIN,RB,25,14,14,0,0,0,0,0,312,1557,4.99,16,54,44,361,8.20,1,5,3,17,3,,294,337.8,346.8,315.8,164,3,3
3,Davante Adams*+,GNB,WR,28,14,14,0,0,0,0,0,0,0,,0,149,115,1374,11.95,18,1,1,18,,,243,358.4,362.4,300.9,117,1,4
4,Travis Kelce*+,KAN,TE,31,15,15,1,2,4,0,0,0,0,,0,145,105,1416,13.49,11,1,1,11,1,,208,312.8,316.8,260.3,117,1,5


In [141]:
next_year_PPR.head()

,Player,PPR
0,Jonathan Taylor*+,373.1
1,Cooper Kupp*+,439.5
2,Deebo Samuel*+,339.0
3,Josh Allen,402.6
4,Austin Ekeler,343.8


Clean up names in the data frame

In [142]:
df.Player = df.Player.str.replace("+", "")
df.Player = df.Player.str.replace("*", "")
next_year_PPR.Player = df.Player.str.replace("+", "")
next_year_PPR.Player = df.Player.str.replace("*", "")


C:\Users\grays\AppData\Local\Temp\ipykernel_29920\4210111499.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.Player = df.Player.str.replace("+", "")
C:\Users\grays\AppData\Local\Temp\ipykernel_29920\4210111499.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.Player = df.Player.str.replace("*", "")
C:\Users\grays\AppData\Local\Temp\ipykernel_29920\4210111499.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  next_year_PPR.Player = df.Player.str.replace("+", "")
C:\Users\grays\AppData\Local\Temp\ipykernel_299

In [143]:
next_year_PPR["Player_cat"] = pd.Categorical(
    next_year_PPR["Player"],
    categories=df["Player"],
    ordered=True
)

df

,Player,Tm,FantPos,Age,G,GS,Cmp,Att,Yds,TD,Int,Att,Yds,Y/A,TD,Tgt,Rec,Yds,Y/R,TD,Fmb,FL,TD,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank
0,Derrick Henry,TEN,RB,26,16,16,0,0,0,0,0,378,2027,5.36,17,31,19,114,6.00,0,3,2,17,1,,314,333.1,341.1,323.6,184,1,1
1,Alvin Kamara,NOR,RB,25,15,10,0,0,0,0,0,187,932,4.98,16,107,83,756,9.11,5,1,0,21,,,295,377.8,383.8,336.3,165,2,2
2,Dalvin Cook,MIN,RB,25,14,14,0,0,0,0,0,312,1557,4.99,16,54,44,361,8.20,1,5,3,17,3,,294,337.8,346.8,315.8,164,3,3
3,Davante Adams,GNB,WR,28,14,14,0,0,0,0,0,0,0,,0,149,115,1374,11.95,18,1,1,18,,,243,358.4,362.4,300.9,117,1,4
4,Travis Kelce,KAN,TE,31,15,15,1,2,4,0,0,0,0,,0,145,105,1416,13.49,11,1,1,11,1,,208,312.8,316.8,260.3,117,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
674,Dwayne Harris,CHI,,33,3,0,0,0,0,0,0,0,0,,0,0,0,0,,0,1,1,0,,,-2,-2.0,-1.0,-2.0,,255,
675,Nate Sudfeld,PHI,QB,27,1,0,5,12,32,0,1,2,12,6.00,0,0,0,0,,0,1,1,0,,,-2,-1.5,0.5,-0.5,,82,
676,Nsimba Webster,LAR,,24,16,0,0,0,0,0,0,0,0,,0,0,0,0,,0,3,1,0,,,-2,-2.0,-1.0,-2.0,,256,
677,Dede Westbrook,JAX,WR,27,2,0,0,0,0,0,0,0,0,,0,1,1,4,4.00,0,1,1,0,,,-2,-0.6,0.4,-1.1,,254,


In [144]:
next_year_PPR["repeat"] = next_year_PPR["Player"].isin(df["Player"])
df["repeat"] = df["Player"].isin(next_year_PPR["Player"])
next_year_PPR = next_year_PPR.loc[next_year_PPR.repeat, :]
next_year_PPR

,Player,PPR,Player_cat,repeat
0,Derrick Henry,373.1,Derrick Henry,True
1,Alvin Kamara,439.5,Alvin Kamara,True
2,Dalvin Cook,339.0,Dalvin Cook,True
3,Davante Adams,402.6,Davante Adams,True
4,Travis Kelce,343.8,Travis Kelce,True
...,...,...,...,...
674,Dwayne Harris,,Dwayne Harris,True
675,Nate Sudfeld,-0.3,Nate Sudfeld,True
676,Nsimba Webster,,Nsimba Webster,True
677,Dede Westbrook,,Dede Westbrook,True


In [145]:
df["Next_Year_PPR"] = next_year_PPR["PPR"]
df.replace("", 0, inplace=True)
df

,Player,Tm,FantPos,Age,G,GS,Cmp,Att,Yds,TD,Int,Att,Yds,Y/A,TD,Tgt,Rec,Yds,Y/R,TD,Fmb,FL,TD,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank,repeat,Next_Year_PPR
0,Derrick Henry,TEN,RB,26,16,16,0,0,0,0,0,378,2027,5.36,17,31,19,114,6.00,0,3,2,17,1,0,314,333.1,341.1,323.6,184,1,1,True,373.1
1,Alvin Kamara,NOR,RB,25,15,10,0,0,0,0,0,187,932,4.98,16,107,83,756,9.11,5,1,0,21,0,0,295,377.8,383.8,336.3,165,2,2,True,439.5
2,Dalvin Cook,MIN,RB,25,14,14,0,0,0,0,0,312,1557,4.99,16,54,44,361,8.20,1,5,3,17,3,0,294,337.8,346.8,315.8,164,3,3,True,339.0
3,Davante Adams,GNB,WR,28,14,14,0,0,0,0,0,0,0,0,0,149,115,1374,11.95,18,1,1,18,0,0,243,358.4,362.4,300.9,117,1,4,True,402.6
4,Travis Kelce,KAN,TE,31,15,15,1,2,4,0,0,0,0,0,0,145,105,1416,13.49,11,1,1,11,1,0,208,312.8,316.8,260.3,117,1,5,True,343.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
674,Dwayne Harris,CHI,0,33,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,-2,-2.0,-1.0,-2.0,0,255,0,True,0
675,Nate Sudfeld,PHI,QB,27,1,0,5,12,32,0,1,2,12,6.00,0,0,0,0,0,0,1,1,0,0,0,-2,-1.5,0.5,-0.5,0,82,0,True,-0.3
676,Nsimba Webster,LAR,0,24,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,1,0,0,0,-2,-2.0,-1.0,-2.0,0,256,0,True,0
677,Dede Westbrook,JAX,WR,27,2,0,0,0,0,0,0,0,0,0,0,1,1,4,4.00,0,1,1,0,0,0,-2,-0.6,0.4,-1.1,0,254,0,True,0


Finally, we fit the data...

In [146]:
reg = linear_model.LinearRegression()
#reg.fit(df[["Age", "G", "GS", "Cmp", "Att", "Yds", "TD", "Int", "Att", "Yds", "Y/A", "TD", "Tgt", "Rec", "Yds", "Y/R", "TD", "Fmb", "FL", "TD", "PPR", "OvRank"]], df["Next_Year_PPR"])
reg.fit(df.drop(columns=["Player", "Tm", "FantPos", "repeat", "Next_Year_PPR"]), df["Next_Year_PPR"])


LinearRegression()

Get R^2 for training data. 0.95 is very promising, but the dataset needs cleaned.

In [147]:
reg.score(df.drop(columns=["Player", "Tm", "FantPos", "repeat", "Next_Year_PPR"]), df["Next_Year_PPR"])

0.9527979064016348

Get prediction matrix.

In [156]:
df_2021 = get_player_df(2021)
df_2021.replace("", 0, inplace=True)
df_2021.dropna(inplace=True)

arr = reg.predict(df_2021.drop(["Player", "Tm", "FantPos"], axis=1))


First-order muliple linear regression projection for 2022 based on 2021 season.

In [164]:
prediction = pd.DataFrame()
prediction["Player"] = df_2021["Player"]
prediction["PPR"] = arr
prediction

prediction.to_csv("pred.csv")